In [1]:
%cd ../../.
from workflow.modules import *

/cluster/project/beltrao/jjaenes/23.03_DHODH


In [3]:
# Choose method to process / run for both
#method_name_ = 'DiffDock'
method_name_ = 'vina'

def read_diffdock(library_id, pdb_id):
    score_col_ = f'docking_score_{pdb_id}'
    fp_ = f'results/DiffDockRescore/{library_id}+{pdb_id}.tsv'
    #df_ = read_gnina(fp_).query('mode_id == 1')[['ligand_id', 'minimizedAffinity']].rename({'minimizedAffinity': score_col_}, axis=1)
    df_ = pd.read_csv(fp_, sep='\t').rename({'CNN_VS': score_col_}, axis=1)
    df_['ligand_id'] = df_['complex_name'].str.removesuffix(f'+{pdb_id}')
    df_['library_id'] = library_id
    df_[score_col_] = df_[score_col_].fillna(0)
    return df_.drop_duplicates(subset=['ligand_id'], keep=False)[['ligand_id', 'library_id', score_col_]]

def read_smina(library_id, pdb_id):
    score_col_ = f'docking_score_{pdb_id}'
    fp_ = f'results/smina/{library_id}+{pdb_id}.sdf'
    df_ = read_gnina(fp_).query('mode_id == 1')[['ligand_id', 'minimizedAffinity']].rename({'minimizedAffinity': score_col_}, axis=1)
    df_[score_col_] = -df_[score_col_]
    df_['library_id'] = library_id
    return df_

if method_name_ == 'DiffDock':
    df_ = pd.concat([
        pd.concat(map(lambda l: read_diffdock(l, '1d3g'), library_id), axis=0).set_index(['ligand_id', 'library_id']),
        pd.concat(map(lambda l: read_diffdock(l, '1d3h'), library_id), axis=0).set_index(['ligand_id', 'library_id']),
        pd.concat(map(lambda l: read_diffdock(l, '4igh'), library_id), axis=0).set_index(['ligand_id', 'library_id']),
        pd.concat(map(lambda l: read_diffdock(l, '6j3c'), library_id), axis=0).set_index(['ligand_id', 'library_id']),
    ], axis=1).reset_index()
elif method_name_ == 'vina':
    df_ = pd.concat([
        pd.concat(map(lambda l: read_smina(l, '1d3g'), library_id), axis=0).set_index(['ligand_id', 'library_id']),
        pd.concat(map(lambda l: read_smina(l, '1d3h'), library_id), axis=0).set_index(['ligand_id', 'library_id']),
        pd.concat(map(lambda l: read_smina(l, '4igh'), library_id), axis=0).set_index(['ligand_id', 'library_id']),
        pd.concat(map(lambda l: read_smina(l, '6j3c'), library_id), axis=0).set_index(['ligand_id', 'library_id']),
    ], axis=1).reset_index()
else:
    assert False

df_.to_csv(f'results/docking_scores_{method_name_}.tsv', sep='\t', index=False, header=True)